In [3]:
!pip install transformers
!pip install seqeval
!pip install seaborn

     ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- ----------------------------------- 30.7/294.6 kB ? eta -:--:--
     ---- -----------------------------

In [3]:
import pandas as pd
import ast
import re
import string
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import transformers
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from seqeval.metrics import f1_score, accuracy_score
import torch.nn as nn



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
"""
def read_file(filepath):
    sentences = []
    sentence = []

    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # Non-empty line
                word, tag = line.split('\t')
                sentence.append((word, tag))
            else:  # Empty line indicating the end of a sentence
                if sentence:  # Add the sentence if it's not empty
                    sentences.append(sentence)
                    sentence = []
        if sentence:  # Add the last sentence if the file doesn't end with a newline
            sentences.append(sentence)
    return sentences

# Example usage
file_path = '/content/drive/My Drive/ijnlp.txt'  # Replace with your file path
data = read_file(file_path)
number_of_sentences = len(data)
print("Total number of sentences:", number_of_sentences)


"""

Total number of sentences: 2006


In [4]:
def read_file(filepath):
    sentences = []
    sentence = []

    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # Check for non-empty line
                if '\t' in line:  # Check if line contains a tab character
                    word, tag = line.split('\t')
                    sentence.append((word, tag))
                    if word == '۔':  # Check if the word is the Urdu full stop
                        sentences.append(sentence)
                        sentence = []
                else:
                    print(f"Line skipped: {line}")  # Optionally, print the problematic line
            # No 'else' block needed for empty lines as they are not expected in this dataset format

    # Add the last sentence if not empty
    if sentence:
        sentences.append(sentence)

    return sentences

# Example usage
file_path = './HUT_COMSATS.txt'  # Replace with your file path
data = read_file(file_path)
number_of_sentences = len(data)
print("Total number of sentences:", number_of_sentences)


Total number of sentences: 5051


In [5]:
sentences = []
labels = []

for sentence in data:
    word_list = [word for word, tag in sentence]
    tag_list = [tag for word, tag in sentence]

    sentences.append(word_list)
    labels.append(tag_list)

# Now `sentences` contains lists of words, and `labels` contains lists of tags
print("Sentences:", sentences[0])
print("Labels:", labels[0])

Sentences: ['کامسیٹس', 'یونیورسٹی', 'اسلام آباد', 'میں', 'پڑھتا', 'ہوں', '۔']
Labels: ['Organization', 'Organization', 'Organization', 'Other', 'Other', 'Other', 'Other']


In [6]:

def flatten_data(data):
    # Flattens the list of sentences (which are lists of (word, tag) tuples) into a list of tuples
    return [(word, tag) for sentence in data for word, tag in sentence]

# Assuming 'data' is loaded from read_file function
flattened_data = flatten_data(data)

# Convert flattened data to a DataFrame
df_train = pd.DataFrame(flattened_data, columns=['Word', 'Tag'])

print(df_train.head())

         Word           Tag
0     کامسیٹس  Organization
1   یونیورسٹی  Organization
2  اسلام آباد  Organization
3         میں         Other
4       پڑھتا         Other


In [7]:
df_train['Tag'].value_counts()

Other           167618
Person            7331
Organization      5778
Number            5376
Location          5184
Date              3007
Designation       2877
Time              1047
Currency           715
Month              154
Year               111
Day                 13
Name: Tag, dtype: int64

In [8]:
tag_values = list(set(df_train["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [10]:
#save tag_values 
import json
with open('tag_values.json', 'w') as fp:
    json.dump(tag_values, fp)

In [8]:
MAX_LEN = 343
bs = 32

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

0

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [12]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []
  for word, label in zip(sentence, text_labels):

      # Tokenize the word and count # of subwords the word is broken into
      tokenized_word = tokenizer.tokenize(word)
      n_subwords = len(tokenized_word)

      # Add the tokenized word to the final tokenized word list
      tokenized_sentence.extend(tokenized_word)

      # Add the same label to the new list of labels `n_subwords` times
      labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [13]:

tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

In [14]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [17]:
labels

[['Organization',
  'Organization',
  'Organization',
  'Organization',
  'Organization',
  'Organization',
  'Organization',
  'Other',
  'Other',
  'Other',
  'Other',
  'Other',
  'Other'],
 ['Person',
  'Person',
  'Person',
  'Other',
  'Other',
  'Other',
  'Other',
  'Other',
  'Other',
  'Person',
  'Person',
  'Person',
  'Other',
  'Person',
  'Person',
  'Person',
  'Person',
  'Person',
  'Other',
  'Person',
  'Person',
  'Person',
  'Location',
  'Other',
  'Person',
  'Person',
  'Person',
  'Person',
  'Other',
  'Other'],
 ['Other',
  'Other',
  'Location',
  'Location',
  'Location',
  'Other',
  'Location',
  'Location',
  'Location',
  'Other',
  'Location',
  'Location',
  'Other',
  'Location',
  'Location',
  'Location',
  'Location',
  'Other',
  'Other',
  'Other',
  'Other',
  'Location',
  'Location',
  'Location',
  'Location',
  'Location',
  'Other',
  'Other',
  'Other'],
 ['Other',
  'Location',
  'Location',
  'Location',
  'Location',
  'Other',
  'Oth

In [15]:

input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [16]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [17]:

attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [18]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=40, test_size=0.2)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=40, test_size=0.2)
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [19]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

C:\Windows\Temp\ipykernel_18968\469025060.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tr_inputs = torch.tensor(tr_inputs)
C:\Windows\Temp\ipykernel_18968\469025060.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_inputs = torch.tensor(val_inputs)
C:\Windows\Temp\ipykernel_18968\469025060.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tr_tags = torch.tensor(tr_tags)
C:\Windows\Temp\ipykernel_18968\469025060.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sour

In [20]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [21]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [27]:
model.cuda();

AssertionError: Torch not compiled with CUDA enabled

In [28]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)


c:\Users\Taimoor Khan\anaconda3\envs\MachineLearningEnv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
from transformers import get_linear_schedule_with_warmup

epochs = 5
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []
#loss_fn = nn.CrossEntropyLoss(weight=weights.cuda())
for _ in trange(epochs, desc="Epoch"):
  # ========================================
  #               Training
  # ========================================
  # Perform one full pass over the training set.

  # Put the model into training mode.
  model.train()
  # Reset the total loss for this epoch.
  total_loss = 0

  # Training loop
  for step, batch in enumerate(train_dataloader):
    # add batch to gpu
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # Always clear any previously calculated gradients before performing a backward pass.
    model.zero_grad()
    # forward pass
    # This will return the loss (rather than the model output)
    # because we have provided the `labels`.
    outputs = model(b_input_ids, token_type_ids=None,
                    attention_mask=b_input_mask,labels=b_labels)

    loss = outputs[0]
    loss.backward()
    # track train loss
    total_loss += loss.item()
    # Clip the norm of the gradient
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    # update parameters
    optimizer.step()
    # Update the learning rate.
    scheduler.step()

  # Calculate the average loss over the training data.
  avg_train_loss = total_loss / len(train_dataloader)
  print("\nAverage train loss: {}".format(avg_train_loss))

  # Store the loss value for plotting the learning curve.
  loss_values.append(avg_train_loss)


  # ========================================
  #               Validation
  # ========================================
  # After the completion of each training epoch, measure our performance on
  # our validation set.

  # Put the model into evaluation mode
  model.eval()
  # Reset the validation loss for this epoch.
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions , true_labels = [], []
  for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients,
    # saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have not provided labels.
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask, labels=b_labels)
    # Move logits and labels to CPU
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences.
    eval_loss += outputs[0].mean().item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)

  eval_loss = eval_loss / len(valid_dataloader)
  validation_loss_values.append(eval_loss)
  print("\nValidation loss: {}".format(eval_loss))
  pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
  valid_tags = [tag_values[l_i] for l in true_labels
                                for l_i in l if tag_values[l_i] != "PAD"]
  print("\nValidation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print()


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]


Average train loss: 0.287827841702878


Epoch:  20%|██        | 1/5 [01:17<05:10, 77.74s/it]


Validation loss: 0.09004908369388431

Validation Accuracy: 0.9495120769603432


Average train loss: 0.06796153036393518


Epoch:  40%|████      | 2/5 [02:37<03:57, 79.08s/it]

...


Validation loss: 0.05240688705816865

Validation Accuracy: 0.9739255311786282


Average train loss: 0.028829090883882028


Epoch:  80%|████████  | 4/5 [05:17<01:19, 79.62s/it]


Validation loss: 0.048753846320323646

Validation Accuracy: 0.9777147207419199


Average train loss: 0.022743979091279384


Epoch: 100%|██████████| 5/5 [06:37<00:00, 79.45s/it]


Validation loss: 0.046848835016135126

Validation Accuracy: 0.978372205688975



In [ ]:
model_path = "/content/drive/My Drive/Dataset/Ubert_model.h5"

# Save the model's state dictionary to the specified path
torch.save(model.state_dict(), model_path)

In [ ]:
def predict_tags_with_tokens(sentence, model, tokenizer, max_len, device):
    # Tokenize the sentence
    inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=max_len, padding='max_length', return_tensors='pt', truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Model in evaluation mode
    model.eval()

    # No gradient calculation
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Convert logits to predicted tag indices
    logits = outputs.logits
    tag_indices = torch.argmax(logits, dim=2)[0].cpu().numpy()  # Take the first sentence

    # Convert input ids to tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())

    # Pair tokens with their predicted tags
    token_tag_pairs = []
    for token, tag_idx in zip(tokens, tag_indices):
        if token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            tag = tag_values[tag_idx]  # Assuming tag_values list is accessible here
            token_tag_pairs.append((token, tag))

    return token_tag_pairs

# Example usage
sentence = "ڈاکٹر ایملی سمتھ، کوانٹم ٹیکنالوجیز کی سی ای او، جانی سنز، ڈینی ڈینیئلز اور ارمغان علی کے ساتھ ورلڈ ٹیک کانفرنس 2023 میں شریک ہوں گی، جو کہ 10 اپریل کو صبح 9:00 بجے حضرو میں منعقد ہوگی۔ اس کانفرنس میں وہ 5 دیگر اعلیٰ ایگزیکٹوز کے ہمراہ ہوں گی۔"
predicted_token_tags = predict_tags_with_tokens(sentence, model, tokenizer, MAX_LEN , device)
for token, tag in predicted_token_tags:
    print(f"('{token}', '{tag}')")

('ڈاکٹر', 'Designation')
('ایم', 'Person')
('##لی', 'Person')
('سمت', 'Person')
('##ھ', 'Person')
('،', 'Other')
('کو', 'Other')
('##ان', 'Other')
('##ٹ', 'Other')
('##م', 'Other')
('ٹ', 'Other')
('##یک', 'Other')
('##نال', 'Other')
('##وج', 'Other')
('##یز', 'Other')
('کی', 'Other')
('سی', 'Other')
('ای', 'Other')
('او', 'Other')
('،', 'Other')
('جان', 'Person')
('##ی', 'Person')
('سن', 'Person')
('##ز', 'Person')
('،', 'Other')
('ڈی', 'Person')
('##نی', 'Person')
('ڈی', 'Person')
('##نی', 'Person')
('##ئل', 'Person')
('##ز', 'Person')
('اور', 'Other')
('ا', 'Person')
('##رم', 'Person')
('##غان', 'Person')
('علی', 'Person')
('کے', 'Other')
('ساتھ', 'Other')
('و', 'Other')
('##رل', 'Other')
('##ڈ', 'Other')
('ٹ', 'Other')
('##یک', 'Other')
('کا', 'Other')
('##نف', 'Other')
('##رن', 'Other')
('##س', 'Other')
('202', 'Date')
('##3', 'Date')
('میں', 'Other')
('ش', 'Other')
('##ریک', 'Other')
('ہوں', 'Other')
('گ', 'Other')
('##ی', 'Other')
('،', 'Other')
('جو', 'Other')
('کہ', 'Other')
('

In [ ]:
def group_subwords(token_tag_pairs):
    grouped_pairs = []
    current_word, current_tag = None, None

    for token, tag in token_tag_pairs:
        if token.startswith("##"):
            # Append subword token to the current word
            current_word += token[2:]
        else:
            # Add the previous word (if any) to the result
            if current_word:
                grouped_pairs.append((current_word, current_tag))
            current_word, current_tag = token, tag

    # Don't forget to add the last word
    if current_word:
        grouped_pairs.append((current_word, current_tag))

    return grouped_pairs

grouped_token_tags = group_subwords(predicted_token_tags)
for word, tag in grouped_token_tags:
    print(f"('{word}', '{tag}')")

('ڈاکٹر', 'Designation')
('ایملی', 'Person')
('سمتھ', 'Person')
('،', 'Other')
('کوانٹم', 'Other')
('ٹیکنالوجیز', 'Other')
('کی', 'Other')
('سی', 'Other')
('ای', 'Other')
('او', 'Other')
('،', 'Other')
('جانی', 'Person')
('سنز', 'Person')
('،', 'Other')
('ڈینی', 'Person')
('ڈینیئلز', 'Person')
('اور', 'Other')
('ارمغان', 'Person')
('علی', 'Person')
('کے', 'Other')
('ساتھ', 'Other')
('ورلڈ', 'Other')
('ٹیک', 'Other')
('کانفرنس', 'Other')
('2023', 'Date')
('میں', 'Other')
('شریک', 'Other')
('ہوں', 'Other')
('گی', 'Other')
('،', 'Other')
('جو', 'Other')
('کہ', 'Other')
('10', 'Date')
('اپریل', 'Date')
('کو', 'Other')
('صبح', 'Time')
('9', 'Time')
(':', 'Time')
('00', 'Time')
('بجے', 'Time')
('حضرو', 'Location')
('میں', 'Other')
('منعقد', 'Other')
('ہوگی', 'Other')
('۔', 'Other')
('اس', 'Other')
('کانفرنس', 'Other')
('میں', 'Other')
('وہ', 'Other')
('5', 'Number')
('دیگر', 'Other')
('اعلیٰ', 'Other')
('ایگزیکٹوز', 'Other')
('کے', 'Other')
('ہمراہ', 'Other')


In [ ]:
def merge_same_tags(token_tag_pairs):
    merged_pairs = []
    current_word, current_tag = None, None

    for word, tag in token_tag_pairs:
        if tag == current_tag:
            # Append word to the current word
            current_word += ' ' + word
        else:
            # Add the previous word (if any) to the result
            if current_word:
                merged_pairs.append((current_word, current_tag))
            current_word, current_tag = word, tag

    # Don't forget to add the last word
    if current_word:
        merged_pairs.append((current_word, current_tag))

    return merged_pairs


merged_token_tags = merge_same_tags(grouped_token_tags)
for word, tag in merged_token_tags:
    print(f"('{word}', '{tag}')")



('ڈاکٹر', 'Designation')
('ایملی سمتھ', 'Person')
('، کوانٹم ٹیکنالوجیز کی سی ای او ،', 'Other')
('جانی سنز', 'Person')
('،', 'Other')
('ڈینی ڈینیئلز', 'Person')
('اور', 'Other')
('ارمغان علی', 'Person')
('کے ساتھ ورلڈ ٹیک کانفرنس', 'Other')
('2023', 'Date')
('میں شریک ہوں گی ، جو کہ', 'Other')
('10 اپریل', 'Date')
('کو', 'Other')
('صبح 9 : 00 بجے', 'Time')
('حضرو', 'Location')
('میں منعقد ہوگی ۔ اس کانفرنس میں وہ', 'Other')
('5', 'Number')
('دیگر اعلیٰ ایگزیکٹوز کے ہمراہ', 'Other')


In [ ]:
def process_text(sentence, model, tokenizer, max_len, device):
    # Tokenize and predict tags
    inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=max_len, padding='max_length', return_tensors='pt', truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    tag_indices = torch.argmax(logits, dim=2)[0].cpu().numpy()

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
    token_tag_pairs = [(token, tag_values[tag_idx]) for token, tag_idx in zip(tokens, tag_indices) if token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]]

    # Group subwords and merge same tags
    grouped_pairs, merged_pairs = [], []
    current_word, current_tag = None, None

    for token, tag in token_tag_pairs:
        # Group subwords
        if token.startswith("##"):
            current_word += token[2:]
        else:
            if current_word:
                grouped_pairs.append((current_word, current_tag))
            current_word, current_tag = token, tag

        # Merge same tags
        if current_tag == tag:
            current_word += ' ' + token
        else:
            if current_word:
                merged_pairs.append((current_word, current_tag))
            current_word, current_tag = token, tag

    if current_word:
        grouped_pairs.append((current_word, current_tag))
        merged_pairs.append((current_word, current_tag))

    return grouped_pairs, merged_pairs


In [1]:
import torch

# Check if CUDA (GPU support) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Taimoor Khan\anaconda3\envs\MachineLearningEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

device(type='cpu')